In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

FILE_PATH = "/content/drive/MyDrive/MobiFall_Dataset_v2.0"


print("🔍 Checking file path:")
print(FILE_PATH)


if os.path.exists(FILE_PATH):
    print(" File exists at this path!")
else:
    print(" File NOT found at this path!")
    exit()

# 2️⃣ Try loading the CSV
try:
    df = pd.read_csv(FILE_PATH)
    print(" CSV loaded successfully!")
    print(" Shape:", df.shape)
    print(" Columns:", list(df.columns))
except Exception as e:
    print(" Error while loading CSV:")
    print(e)


In [ ]:
!pip install streamlit -q
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.1 MB/s eta 0:00:00
--2025-12-13 06:23:13--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb [following]
--2025-12-13 06:23:13--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/8a32f7c6-649c-4f0d-806d-e14c19d0786d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-13T07%3A06%3A05Z&rscd=attachment%3B+f

In [ ]:
%%writefile app.py
import os
from pathlib import Path
from datetime import datetime, time

import streamlit as st
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
)

import plotly.express as px


MOBIFALL_BASE = "/content/drive/MyDrive/MobiFall_Dataset_v2.0"


st.set_page_config(
    page_title="Elderly Care Monitoring Dashboard",
    layout="wide",
    page_icon="🧓",
)

# Refined CSS
st.markdown(
    """
    <style>
    .main {
        background: linear-gradient(135deg, #f9fafb 0%, #edf2ff 50%, #f9fafb 100%);
    }
    .block-container {
        padding-top: 1.5rem;
        padding-bottom: 1.5rem;
    }
    .metric-card {
        padding: 0.6rem 0.9rem;
        border-radius: 0.9rem;
        background-color: rgba(255,255,255,0.9);
        box-shadow: 0 2px 6px rgba(15, 23, 42, 0.06);
        border: 1px solid #e5e7eb;
        display: flex;
        flex-direction: column;
        gap: 0.15rem;
        min-height: 70px;
    }
    .metric-label {
        font-size: 0.75rem;
        font-weight: 600;
        letter-spacing: 0.08em;
        text-transform: uppercase;
        color: #6b7280;
    }
    .metric-value {
        font-size: 1.4rem;
        font-weight: 700;
        color: #111827;
    }
    .metric-help {
        font-size: 0.75rem;
        color: #9ca3af;
    }
    .section-card {
        padding: 1rem 1.3rem;
        border-radius: 1rem;
        background-color: rgba(255,255,255,0.96);
        box-shadow: 0 2px 10px rgba(15, 23, 42, 0.06);
        border: 1px solid #e5e7eb;
        margin-bottom: 1.1rem;
    }
    .stTabs [data-baseweb="tab-list"] {
        gap: 0.25rem;
    }
    .stTabs [data-baseweb="tab"] {
        border-radius: 999px;
        padding: 0.35rem 0.9rem;
        background-color: #e5e7eb;
    }
    .stTabs [aria-selected="true"] {
        background-color: #4f46e5 !important;
        color: white !important;
    }
    </style>
    """,
    unsafe_allow_html=True,
)

def metric_card(label, value, help_text=None):
    html = f"""
    <div class="metric-card">
        <div class="metric-label">{label}</div>
        <div class="metric-value">{value}</div>
        {f'<div class="metric-help">{help_text}</div>' if help_text else ""}
    </div>
    """
    st.markdown(html, unsafe_allow_html=True)


# --------------------------------------------------
# Helper functions – CSV datasets
# --------------------------------------------------
@st.cache_data
def load_health_data(path: str = "/content/drive/MyDrive/health_monitoring.csv") -> pd.DataFrame:
    df = pd.read_csv(path)

    # Parse Timestamp
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

    # Parse blood pressure "120/80 mmHg" -> systolic/diastolic (for EDA only)
    if "Blood Pressure" in df.columns:
        bp = df["Blood Pressure"].astype(str).str.extract(r"(?P<Systolic>\\d+)\\s*/\\s*(?P<Diastolic>\\d+)")
        df["Systolic_BP"] = pd.to_numeric(bp["Systolic"], errors="coerce")
        df["Diastolic_BP"] = pd.to_numeric(bp["Diastolic"], errors="coerce")

    # Convert Yes/No columns to binary (_bin) for plots
    yes_no_cols = [c for c in df.columns if "(Yes/No)" in c]
    for col in yes_no_cols:
        df[col + "_bin"] = df[col].map({"Yes": 1, "No": 0})

    # Hour of day (used in model)
    df["Hour"] = df["Timestamp"].dt.hour

    return df


@st.cache_data
def load_safety_data(path: str = "/content/drive/MyDrive/safety_monitoring.csv") -> pd.DataFrame:
    df = pd.read_csv(path)
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

    yes_no_cols = [c for c in df.columns if "(Yes/No)" in c]
    for col in yes_no_cols:
        df[col + "_bin"] = df[col].map({"Yes": 1, "No": 0})

    df["Hour"] = df["Timestamp"].dt.hour
    return df


@st.cache_data
def load_daily_data(path: str = "/content/drive/MyDrive/daily_reminder.csv") -> pd.DataFrame:
    df = pd.read_csv(path)
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
    return df

@st.cache_resource
def train_health_model(df: pd.DataFrame):
    """
    Train a Logistic Regression model to predict Alert Triggered (Yes/No)
    based on vital signs.
    """
    df = df.copy()

    target_col_bin = "Alert Triggered (Yes/No)_bin"
    if target_col_bin not in df.columns:
        if "Alert Triggered (Yes/No)" not in df.columns:
            return None, None
        df[target_col_bin] = df["Alert Triggered (Yes/No)"].map({"Yes": 1, "No": 0})

    if "Hour" not in df.columns:
        df["Hour"] = df["Timestamp"].dt.hour

    features = ["Heart Rate", "Glucose Levels", "Oxygen Saturation (SpO₂%)", "Hour"]

    model_df = df.dropna(subset=features + [target_col_bin])
    if model_df.empty or model_df[target_col_bin].nunique() < 2:
        return None, None

    X = model_df[features]
    y = model_df[target_col_bin]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42, stratify=y
    )

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1": f1_score(y_test, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_test, y_proba),
        "confusion_matrix": confusion_matrix(y_test, y_pred),
        "features": features,
    }

    return model, metrics

MOBIFALL_BASE = "/content/drive/MyDrive/MobiFall_Dataset_v2.0"

def load_txt_file_mobifall(path: str) -> np.ndarray:
    """
    Load one MobiFall txt file -> np.array of shape (N, 4): [t, x, y, z]
    """
    data = []
    reached_data = False

    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if line == "@DATA":
                reached_data = True
                continue
            if reached_data and line:
                parts = line.split(",")
                if len(parts) != 4:
                    continue
                data.append([float(x) for x in parts])

    return np.array(data)


def extract_features_mobifall(arr: np.ndarray):
    """
    Same logic as your extract_features():
    builds mag_mean, mag_std, mag_max, mag_min, mag_range, mag_energy, len_seq
    """
    if arr.shape[0] == 0:
        return None

    t = arr[:, 0]
    x = arr[:, 1]
    y = arr[:, 2]
    z = arr[:, 3]

    mag = np.sqrt(x**2 + y**2 + z**2)

    feats = {
        "mag_mean": mag.mean(),
        "mag_std": mag.std(),
        "mag_max": mag.max(),
        "mag_min": mag.min(),
        "mag_range": mag.max() - mag.min(),
        "mag_energy": np.sum(mag**2) / len(mag),
        "len_seq": len(mag),
    }
    return feats


@st.cache_resource
def train_mobifall_rf(base_path: str = MOBIFALL_BASE):
    """
    One-shot function that mirrors your notebook:

    - loop over sub*/FALLS + ADL
    - build feature_rows (mag_mean, mag_std, ...)
    - subject-level train/test split
    - StandardScaler + RandomForestClassifier
    - PCA(2) for visualization
    """
    if not os.path.exists(base_path):
        raise FileNotFoundError(f"MobiFall base path not found: {base_path}")

    FALL_ACTS = ["BSC", "FKL", "FOL", "SDL"]
    ADL_ACTS  = ["CSI","CSO","JOG","JUM","SCH","STD","STN","STU","WAL"]

    subjects = sorted([s for s in os.listdir(base_path) if s.startswith("sub")])
    feature_rows = []

    for subj in subjects:
        subj_path = os.path.join(base_path, subj)


        falls_folder = os.path.join(subj_path, "FALLS")
        if os.path.exists(falls_folder):
            for act in FALL_ACTS:
                act_path = os.path.join(falls_folder, act)
                if not os.path.exists(act_path):
                    continue

                files = [f for f in os.listdir(act_path) if f.endswith(".txt")]
                for f in files:
                    arr = load_txt_file_mobifall(os.path.join(act_path, f))
                    feats = extract_features_mobifall(arr)
                    if feats is None:
                        continue
                    feats["subject"] = subj
                    feats["activity"] = act
                    feats["label"] = 1
                    feature_rows.append(feats)

        # ADL
        adl_folder = os.path.join(subj_path, "ADL")
        if os.path.exists(adl_folder):
            for act in ADL_ACTS:
                act_path = os.path.join(adl_folder, act)
                if not os.path.exists(act_path):
                    continue

                files = [f for f in os.listdir(act_path) if f.endswith(".txt")]
                for f in files:
                    arr = load_txt_file_mobifall(os.path.join(act_path, f))
                    feats = extract_features_mobifall(arr)
                    if feats is None:
                        continue
                    feats["subject"] = subj
                    feats["activity"] = act
                    feats["label"] = 0   # NON-FALL
                    feature_rows.append(feats)

    df = pd.DataFrame(feature_rows)

    # subject-level split
    all_subjects = df["subject"].unique()
    train_subj, test_subj = train_test_split(all_subjects, test_size=0.33, random_state=42)

    train_df = df[df.subject.isin(train_subj)]
    test_df  = df[df.subject.isin(test_subj)]

    feature_cols = [
        "mag_mean",
        "mag_std",
        "mag_max",
        "mag_min",
        "mag_range",
        "mag_energy",
        "len_seq",
    ]

    X_train = train_df[feature_cols]
    X_test  = test_df[feature_cols]
    y_train = train_df["label"]
    y_test  = test_df["label"]

    scaler = StandardScaler()
    X_train_sc = scaler.fit_transform(X_train)
    X_test_sc  = scaler.transform(X_test)

    rf = RandomForestClassifier(
        n_estimators=300,
        max_depth=12,
        class_weight="balanced",
        random_state=42,
    )
    rf.fit(X_train_sc, y_train)
    preds = rf.predict(X_test_sc)

    cm = confusion_matrix(y_test, preds)
    metrics = {
        "accuracy": accuracy_score(y_test, preds),
        "precision": precision_score(y_test, preds, zero_division=0),
        "recall": recall_score(y_test, preds, zero_division=0),
        "f1": f1_score(y_test, preds, zero_division=0),
        "cm": cm,
        "n_train": len(train_df),
        "n_test": len(test_df),
        "feature_cols": feature_cols,
    }

    # 2D PCA visualization
    X_all = df[feature_cols]
    X_all_sc = scaler.fit_transform(X_all)
    pca = PCA(n_components=2)
    pcs = pca.fit_transform(X_all_sc)
    pca_df = pd.DataFrame(
        {"pc1": pcs[:, 0], "pc2": pcs[:, 1], "label": df["label"].values}
    )

    return df, rf, scaler, metrics, pca_df


# Metric card helper (custom UI, no ugly bars)

def metric_card(label, value, help_text=None):
    html = f"""
    <div class="metric-card">
        <div class="metric-label">{label}</div>
        <div class="metric-value">{value}</div>
    """
    if help_text:
        html += f'<div class="metric-help">{help_text}</div>'
    html += "</div>"
    st.markdown(html, unsafe_allow_html=True)

# Load CSV-based data

try:
    health_df = load_health_data()
    safety_df = load_safety_data()
    daily_df = load_daily_data()
except FileNotFoundError as e:
    st.error(
        "Could not find one or more CSV files. "
        "Make sure `health_monitoring.csv`, `safety_monitoring.csv`, and `daily_reminder.csv` "
        "are in the expected Google Drive location or update the paths in load_* functions."
    )
    st.stop()

health_model, health_metrics = train_health_model(health_df)


# Sidebar navigation

st.sidebar.title("Elderly Care Companion")
st.sidebar.markdown(
    """
This app is designed for **families, caregivers, and clinicians** to:

- keep an eye on vital signs
- understand fall risk patterns
- manage daily reminders for medication & routines
- run simple prediction demos
"""
)

page = st.sidebar.radio(
    "Go to section",
    ["Overview", "Health Monitoring", "Safety Monitoring", "Reminders & Adherence", "Alert Prediction"],
)

st.sidebar.markdown("---")
st.sidebar.caption("Data source: AI for Elderly Care & Support")


# Overview Page

if page == "Overview":
    st.markdown("## Elderly Care Monitoring Dashboard")
    st.markdown(
        """
This dashboard gives a **single view of an older adult’s day**:
health, safety, and reminders – all in one place.
"""
    )

    col1, col2, col3, col4 = st.columns(4)
    with col1:
        metric_card("Total Health Records", f"{len(health_df):,}")
    with col2:
        metric_card("Total Safety Events", f"{len(safety_df):,}")
    with col3:
        metric_card("Total Reminders", f"{len(daily_df):,}")
    with col4:
        alert_rate = (
            health_df["Alert Triggered (Yes/No)"].eq("Yes").mean() * 100
            if "Alert Triggered (Yes/No)" in health_df.columns
            else np.nan
        )
        metric_card("Health Alert Rate", f"{alert_rate:.1f}%" if not np.isnan(alert_rate) else "N/A")

    st.markdown("### High-level Trends")
    tab1, tab2, tab3 = st.tabs(["Health Alerts over Time", "Falls over Time", "Reminder Acknowledgement"])

    with tab1:
        df = health_df.dropna(subset=["Timestamp"])
        if not df.empty and "Alert Triggered (Yes/No)_bin" in df.columns:
            health_daily = (
                df.set_index("Timestamp")
                .resample("D")["Alert Triggered (Yes/No)_bin"]
                .mean()
                .reset_index()
            )
            fig = px.line(
                health_daily,
                x="Timestamp",
                y="Alert Triggered (Yes/No)_bin",
                labels={"Alert Triggered (Yes/No)_bin": "Alert Rate"},
                title="Daily Health Alert Rate",
            )
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.info("No timestamp data available for health monitoring.")

    with tab2:
        df = safety_df.dropna(subset=["Timestamp"])
        if not df.empty and "Fall Detected (Yes/No)_bin" in df.columns:
            safety_daily = (
                df.set_index("Timestamp")
                .resample("D")["Fall Detected (Yes/No)_bin"]
                .sum()
                .reset_index()
            )
            fig = px.bar(
                safety_daily,
                x="Timestamp",
                y="Fall Detected (Yes/No)_bin",
                labels={"Fall Detected (Yes/No)_bin": "Number of Falls"},
                title="Daily Count of Detected Falls",
            )
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.info("No timestamp data available for safety monitoring.")

    with tab3:
        df = daily_df.copy()
        if not df.empty:
            df["Acknowledged_bin"] = df["Acknowledged (Yes/No)"].map({"Yes": 1, "No": 0})
            ack_rate = df["Acknowledged_bin"].mean() * 100
            fig = px.histogram(
                df,
                x="Reminder Type",
                color="Acknowledged (Yes/No)",
                barmode="group",
                title=f"Reminder Acknowledgement by Type (Overall ack: {ack_rate:.1f}%)",
            )
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.info("No daily reminder data available.")


# Health Monitoring Page

elif page == "Health Monitoring":
    st.markdown("## Health Monitoring")
    st.markdown(
        "Use this view to **scan vital signs and alerts** for a monitored person over time."
    )

    with st.container():
        st.markdown('<div class="section-card">', unsafe_allow_html=True)

        col1, col2, col3, col4 = st.columns(4)
        with col1:
            metric_card("Records", f"{len(health_df):,}")
        with col2:
            rate = health_df["Alert Triggered (Yes/No)"].eq("Yes").mean() * 100
            metric_card("Alert Triggered", f"{rate:.1f}%")
        with col3:
            rate_cg = health_df["Caregiver Notified (Yes/No)"].eq("Yes").mean() * 100
            metric_card("Caregiver Notified", f"{rate_cg:.1f}%")
        with col4:
            devices = health_df["Device-ID/User-ID"].nunique()
            metric_card("Devices / Users", f"{devices:,}")

        st.markdown("</div>", unsafe_allow_html=True)

    # Filters
    st.markdown('<div class="section-card">', unsafe_allow_html=True)
    st.markdown("### Filter data for a specific person or period")
    c1, c2, c3 = st.columns(3)

    with c1:
        device_filter = st.multiselect(
            "Device / User",
            options=sorted(health_df["Device-ID/User-ID"].unique()),
            default=[],
        )
    with c2:
        min_ts = health_df["Timestamp"].min()
        max_ts = health_df["Timestamp"].max()
        date_range = st.date_input(
            "Date range",
            value=(min_ts.date() if pd.notnull(min_ts) else datetime.today().date(),
                   max_ts.date() if pd.notnull(max_ts) else datetime.today().date()),
        )
    with c3:
        alert_filter = st.selectbox(
            "Alert triggered?",
            options=["Both", "Yes", "No"],
        )

    df = health_df.copy()
    if device_filter:
        df = df[df["Device-ID/User-ID"].isin(device_filter)]

    if isinstance(date_range, (list, tuple)) and len(date_range) == 2:
        start_date, end_date = date_range
        df = df[
            (df["Timestamp"].dt.date >= start_date)
            & (df["Timestamp"].dt.date <= end_date)
        ]

    if alert_filter != "Both":
        df = df[df["Alert Triggered (Yes/No)"] == alert_filter]

    st.markdown("</div>", unsafe_allow_html=True)

    # Tabs for EDA
    st.markdown('<div class="section-card">', unsafe_allow_html=True)
    tab1, tab2, tab3 = st.tabs(["Vital Distributions", "Alerts vs Vitals", "Time Series (per Device)"])

    with tab1:
        col_a, col_b, col_c = st.columns(3)
        with col_a:
            fig = px.histogram(df, x="Heart Rate", nbins=40, title="Heart Rate Distribution")
            st.plotly_chart(fig, use_container_width=True)
        with col_b:
            fig = px.histogram(df, x="Glucose Levels", nbins=40, title="Glucose Levels Distribution")
            st.plotly_chart(fig, use_container_width=True)
        with col_c:
            fig = px.histogram(
                df,
                x="Oxygen Saturation (SpO₂%)",
                nbins=40,
                title="SpO₂ Distribution",
            )
            st.plotly_chart(fig, use_container_width=True)

    with tab2:
        hover_cols = ["Glucose Levels"]
        if "Blood Pressure" in df.columns:
            hover_cols.append("Blood Pressure")

        fig = px.scatter(
            df,
            x="Heart Rate",
            y="Oxygen Saturation (SpO₂%)",
            color="Alert Triggered (Yes/No)",
            title="Alerts vs Heart Rate & SpO₂",
            hover_data=hover_cols,
        )
        st.plotly_chart(fig, use_container_width=True)

        fig = px.box(
            df,
            x="Alert Triggered (Yes/No)",
            y="Glucose Levels",
            title="Glucose Levels by Alert Status",
        )
        st.plotly_chart(fig, use_container_width=True)

    with tab3:
        dev = st.selectbox(
            "Select a Device / User for time series",
            options=sorted(health_df["Device-ID/User-ID"].unique()),
        )
        df_dev = df[df["Device-ID/User-ID"] == dev].sort_values("Timestamp")

        if not df_dev.empty:
            fig = px.line(
                df_dev,
                x="Timestamp",
                y=["Heart Rate", "Glucose Levels", "Oxygen Saturation (SpO₂%)"],
                title=f"Vitals over Time – {dev}",
            )
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.info("No data available for this selection.")

    st.markdown("</div>", unsafe_allow_html=True)


# Safety Monitoring Page

elif page == "Safety Monitoring":
    st.markdown("## Safety Monitoring (Falls & Mobility)")
    st.markdown(
        "Here you can explore **movement patterns, fall events, and a MobiFall-based fall detection model**."
    )

    with st.container():
        st.markdown('<div class="section-card">', unsafe_allow_html=True)

        col1, col2, col3, col4 = st.columns(4)
        with col1:
            metric_card("Safety Events", f"{len(safety_df):,}")
        with col2:
            fall_rate = safety_df["Fall Detected (Yes/No)"].eq("Yes").mean() * 100
            metric_card("Fall Detected", f"{fall_rate:.1f}%")
        with col3:
            alerted_rate = safety_df["Alert Triggered (Yes/No)"].eq("Yes").mean() * 100
            metric_card("Safety Alerts", f"{alerted_rate:.1f}%")
        with col4:
            locations = safety_df["Location"].nunique()
            metric_card("Unique Locations", f"{locations:,}")

        st.markdown("</div>", unsafe_allow_html=True)

    st.markdown('<div class="section-card">', unsafe_allow_html=True)
    tab1, tab2, tab3, tab4 = st.tabs(
        ["Movement & Falls", "Impact & Inactivity", "Location Heat", "Fall Detection Model (MobiFall)"]
    )

    # Existing charts
    with tab1:
        fig = px.histogram(
            safety_df,
            x="Movement Activity",
            color="Fall Detected (Yes/No)",
            barmode="group",
            title="Movement Activity vs Fall Detection",
        )
        st.plotly_chart(fig, use_container_width=True)

    with tab2:
        fig = px.scatter(
            safety_df,
            x="Impact Force Level",
            y="Post-Fall Inactivity Duration (Seconds)",
            color="Fall Detected (Yes/No)",
            title="Impact Force vs Inactivity Duration",
            hover_data=["Location"],
        )
        st.plotly_chart(fig, use_container_width=True)

    with tab3:
        fig = px.histogram(
            safety_df,
            x="Location",
            color="Fall Detected (Yes/No)",
            title="Falls by Location",
        )
        st.plotly_chart(fig, use_container_width=True)

    # New: MobiFall model + prediction demo
    with tab4:
      st.markdown("### MobiFall-based Fall Detection Model")

      try:
          with st.spinner("Building features & training Random Forest on MobiFall data..."):
              features_df, model_fall, scaler_fall, fall_metrics, pca_df = train_mobifall_rf(MOBIFALL_BASE)
      except FileNotFoundError:
          st.warning(
              f"MobiFall dataset not found at path: `{MOBIFALL_BASE}`. "
              "Please mount / unzip the dataset there or update MOBIFALL_BASE in the code."
          )
      else:
          # ---- Model performance cards ----
          colm1, colm2, colm3, colm4 = st.columns(4)
          with colm1:
              metric_card("Accuracy", f"{fall_metrics['accuracy']:.3f}")
          with colm2:
              metric_card("Precision", f"{fall_metrics['precision']:.3f}")
          with colm3:
              metric_card("Recall", f"{fall_metrics['recall']:.3f}")
          with colm4:
              metric_card("F1-score", f"{fall_metrics['f1']:.3f}")

          st.caption(
              f"Train files: {fall_metrics['n_train']}  ·  "
              f"Test files: {fall_metrics['n_test']}"
          )

          # ---- Confusion matrix ----
          cm_fall = fall_metrics["cm"]
          cm_df_fall = pd.DataFrame(
              cm_fall,
              index=["Actual: Non-fall", "Actual: Fall"],
              columns=["Pred: Non-fall", "Pred: Fall"],
          )
          fig_cm = px.imshow(
              cm_df_fall,
              text_auto=True,
              color_continuous_scale="Viridis",
              title="Confusion Matrix – Fall Detection (MobiFall)",
          )
          st.plotly_chart(fig_cm, use_container_width=True)

          # ---- PCA scatter ----
          fig_pca = px.scatter(
              pca_df,
              x="pc1",
              y="pc2",
              color=pca_df["label"].map({0: "Non-fall", 1: "Fall"}),
              title="PCA 2D Visualization of MobiFall Features",
              labels={"color": "Class"},
              opacity=0.7,
          )
          st.plotly_chart(fig_pca, use_container_width=True)

          # ---- User-oriented prediction demo ----
          st.markdown("### Simulate a Custom Scenario")

          st.write(
              "Use the sliders below to simulate a few seconds of motion from a wearable sensor. "
              "The model will estimate whether this pattern looks like a **fall** or **normal activity**."
          )

          feature_cols = [
              "mag_mean",
              "mag_std",
              "mag_max",
              "mag_min",
              "mag_range",
              "mag_energy",
              "len_seq",
          ]

          # Get reasonable ranges from data
          stats = features_df[feature_cols].describe()

          def slider_range(col, integer=False):
              vmin = float(stats.loc["min", col])
              vmax = float(stats.loc["max", col])
              vmed = float(stats.loc["50%", col])
              if integer:
                  return int(vmin), int(vmax), int(vmed)
              return vmin, vmax, vmed

          with st.form("mobifall_demo_form"):
              c1, c2, c3 = st.columns(3)
              with c1:
                  mn_min, mn_max, mn_med = slider_range("mag_mean")
                  mag_mean = st.slider("Avg magnitude", mn_min, mn_max, mn_med)

                  sd_min, sd_max, sd_med = slider_range("mag_std")
                  mag_std = st.slider("Variability (std)", sd_min, sd_max, sd_med)
              with c2:
                  mx_min, mx_max, mx_med = slider_range("mag_max")
                  mag_max = st.slider("Peak magnitude", mx_min, mx_max, mx_med)

                  rg_min, rg_max, rg_med = slider_range("mag_range")
                  mag_range = st.slider("Range", rg_min, rg_max, rg_med)
              with c3:
                  en_min, en_max, en_med = slider_range("mag_energy")
                  mag_energy = st.slider("Signal energy", en_min, en_max, en_med)

                  ls_min, ls_max, ls_med = slider_range("len_seq", integer=True)
                  len_seq = st.slider("Sequence length (samples)",
                                      ls_min, ls_max, ls_med)

              submitted_fall = st.form_submit_button("Predict Fall Risk")

          if submitted_fall:
              sample = pd.DataFrame(
                  [{
                      "mag_mean": mag_mean,
                      "mag_std": mag_std,
                      "mag_max": mag_max,
                      "mag_min": mag_max - mag_range,  # simple consistent relation
                      "mag_range": mag_range,
                      "mag_energy": mag_energy,
                      "len_seq": len_seq,
                  }]
              )
              sample_sc = scaler_fall.transform(sample[feature_cols])
              proba_fall = float(model_fall.predict_proba(sample_sc)[0, 1])
              label = "Fall" if proba_fall >= 0.5 else "Non-fall"

              st.markdown("---")
              st.markdown(f"### Prediction: **{label}**")
              st.progress(proba_fall)
              st.write(f"Estimated probability of **fall**: **{proba_fall * 100:.1f}%**")

              # 🔔 High-risk notification if > 85%
              if proba_fall >= 0.85:
                  st.error(
                      "High fall risk detected. Please seek help immediately or call a family member / caregiver."
                  )
              else:
                  st.info(
                      "All Good!"
                  )


# Reminders & Adherence Page

elif page == "Reminders & Adherence":
    st.markdown("## Daily Reminders & Adherence")
    st.markdown(
        "Use this page to see how well reminders are being followed, and to **mock-up new reminders** "
        "for the person you’re caring for."
    )

    # Session state to hold custom reminders -demo
    if "custom_reminders" not in st.session_state:
        st.session_state["custom_reminders"] = []

    #Reminder customisation form
    st.markdown('<div class="section-card">', unsafe_allow_html=True)
    st.markdown("### Create a Custom Reminder")
    with st.form("custom_reminder_form"):
        c1, c2 = st.columns(2)
        with c1:
            reminder_type = st.selectbox(
                "Reminder Type",
                options=sorted(daily_df["Reminder Type"].unique()),
            )
            reminder_message = st.text_input(
                "What should we remind them about?",
                value="Take morning medicine with water",
            )
        with c2:
            date_val = st.date_input("Date", value=datetime.today().date())
            time_val = st.time_input("Time", value=time(9, 0))
            already_ack = st.checkbox("Mark as already acknowledged", value=False)

        submitted_custom = st.form_submit_button("Add to today's plan")

    if submitted_custom:
        ts = datetime.combine(date_val, time_val)
        new_row = {
            "Timestamp": ts,
            "Reminder Type": reminder_type,
            "Reminder Message": reminder_message,
            "Scheduled Time": ts.time().strftime("%H:%M:%S"),
            "Reminder Sent (Yes/No)": "Yes",
            "Acknowledged (Yes/No)": "Yes" if already_ack else "No",
        }
        st.session_state["custom_reminders"].append(new_row)
        st.success("Custom reminder added for this session.")

    st.markdown("</div>", unsafe_allow_html=True)

    # Combine base data + custom reminders for visuals
    base_df = daily_df.copy()
    custom_df = pd.DataFrame(st.session_state["custom_reminders"])
    if not custom_df.empty:
        df = pd.concat([base_df, custom_df], ignore_index=True, sort=False)
    else:
        df = base_df

    st.markdown('<div class="section-card">', unsafe_allow_html=True)

    df["Ack_bin"] = df["Acknowledged (Yes/No)"].map({"Yes": 1, "No": 0})
    df["Sent_bin"] = df["Reminder Sent (Yes/No)"].map({"Yes": 1, "No": 0})

    col1, col2, col3 = st.columns(3)
    with col1:
        metric_card("Total Reminders (incl. custom)", f"{len(df):,}")
    with col2:
        sent_rate = df["Sent_bin"].mean() * 100
        metric_card("Reminders Sent", f"{sent_rate:.1f}%")
    with col3:
        ack_rate = df["Ack_bin"].mean() * 100
        metric_card("Acknowledged", f"{ack_rate:.1f}%")

    st.markdown("</div>", unsafe_allow_html=True)

    st.markdown('<div class="section-card">', unsafe_allow_html=True)
    tab1, tab2 = st.tabs(["By Reminder Type", "Timeline"])

    with tab1:
        fig = px.histogram(
            df,
            x="Reminder Type",
            color="Acknowledged (Yes/No)",
            barmode="group",
            title="Reminder Acknowledgement by Type",
        )
        st.plotly_chart(fig, use_container_width=True)

    with tab2:
        df_ts = df.dropna(subset=["Timestamp"]).set_index("Timestamp")
        if not df_ts.empty:
            daily_ack = (
                df_ts.resample("D")["Ack_bin"]
                .mean()
                .reset_index()
            )
            fig = px.line(
                daily_ack,
                x="Timestamp",
                y="Ack_bin",
                labels={"Ack_bin": "Acknowledgement Rate"},
                title="Daily Reminder Acknowledgement Rate",
            )
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.info("No timestamp data in reminders.")

    st.markdown("</div>", unsafe_allow_html=True)



# Alert Prediction Demo Page

elif page == "Alert Prediction":
    st.markdown("## Health Alert Prediction")
    st.markdown(
        "This model uses a **Logistic Regression model** trained on the health monitoring "
        "data to estimate the probability of an alert being triggered based on "
        "the patient's current vital signs."
    )

    if health_model is None or health_metrics is None:
        st.warning("Model could not be trained due to missing/invalid data.")
        st.stop()

    col1, col2, col3, col4, col5 = st.columns(5)
    with col1:
        metric_card("Accuracy", f"{health_metrics['accuracy']:.3f}")
    with col2:
        metric_card("Precision", f"{health_metrics['precision']:.3f}")
    with col3:
        metric_card("Recall", f"{health_metrics['recall']:.3f}")
    with col4:
        metric_card("F1-score", f"{health_metrics['f1']:.3f}")
    with col5:
        metric_card("ROC-AUC", f"{health_metrics['roc_auc']:.3f}")

    # Confusion matrix
    st.markdown('<div class="section-card">', unsafe_allow_html=True)
    st.markdown("### Confusion Matrix")
    cm = health_metrics["confusion_matrix"]
    cm_df = pd.DataFrame(
        cm,
        index=["Actual: No Alert", "Actual: Alert"],
        columns=["Pred: No Alert", "Pred: Alert"],
    )
    fig = px.imshow(
        cm_df,
        text_auto=True,
        color_continuous_scale="Blues",
        title="Confusion Matrix – Health Alert Model",
    )
    st.plotly_chart(fig, use_container_width=True)
    st.markdown("</div>", unsafe_allow_html=True)

    # Prediction form
    st.markdown('<div class="section-card">', unsafe_allow_html=True)
    st.markdown("### Simulate a Patient State")

    # Reasonable ranges from data
    hr_min, hr_max = int(health_df["Heart Rate"].min()), int(health_df["Heart Rate"].max())
    glu_min, glu_max = int(health_df["Glucose Levels"].min()), int(health_df["Glucose Levels"].max())
    spo2_min, spo2_max = int(health_df["Oxygen Saturation (SpO₂%)"].min()), int(health_df["Oxygen Saturation (SpO₂%)"].max())

    with st.form("prediction_form"):
        c1, c2 = st.columns(2)
        with c1:
            heart_rate = st.slider(
                "Heart Rate (bpm)",
                min_value=hr_min,
                max_value=hr_max,
                value=int(np.median(health_df["Heart Rate"]))
            )
            glucose = st.slider(
                "Glucose Levels (mg/dL)",
                min_value=glu_min,
                max_value=glu_max,
                value=int(np.median(health_df["Glucose Levels"]))
            )
        with c2:
            spo2 = st.slider(
                "Oxygen Saturation (SpO₂%)",
                min_value=spo2_min,
                max_value=spo2_max,
                value=int(np.median(health_df["Oxygen Saturation (SpO₂%)"]))
            )
            hour = st.slider(
                "Hour of Day",
                min_value=0,
                max_value=23,
                value=10
            )

        submitted = st.form_submit_button("Predict Alert Risk")

    if submitted:
        sample = pd.DataFrame(
            [
                {
                    "Heart Rate": heart_rate,
                    "Glucose Levels": glucose,
                    "Oxygen Saturation (SpO₂%)": spo2,
                    "Hour": hour,
                }
            ]
        )
        proba = float(health_model.predict_proba(sample)[0, 1])
        pred = "Alert" if proba >= 0.5 else "No Alert"

        st.markdown("---")
        st.markdown(f"### Prediction: **{pred}**")
        st.progress(proba)
        st.write(f"Estimated probability of alert being triggered: **{proba * 100:.1f}%**")

        # 🔔 High-risk notification if > 85%
        if proba >= 0.85:
            st.error(
                "⚠️ High health risk detected. Please contact a caregiver or family member immediately."
            )
        else:
            st.info(
                "All Good! Take Care"
            )

        # st.caption(
        #     "Note: This is a simple demo model trained only on this dataset. "
        #     "In real deployments, clinical validation and more robust modeling would be required."
        # )


    st.markdown("</div>", unsafe_allow_html=True)



Writing app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &


In [ ]:
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate


2025-12-13T06:23:19Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-12-13T06:23:19Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-12-13T06:23:21Z INF +--------------------------------------------------------------------------------------------+
2025-12-13T06:23:21Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-12-13T06:23:21Z INF |  https://delete-yrs-grown-honolulu.trycloudflare.com  